1. Write a decorator that ensures a function is only called by users with a specific role. Each function should have an user_type with a string type in kwargs. 

Напишите декоратор, обеспечивающий вызов функции только пользователями с определенной ролью. Каждая функция должна иметь user_type со строковым типом в kwargs.
Example:
@is_admin
def show_customer_receipt(user_type: str):
    # Some very dangerous operation

show_customer_receipt(user_type='user')
> ValueError: Permission denied

show_customer_receipt(user_type='admin')
> function pass as it should be

In [69]:
def is_admin(func):
    def wrapper(**kwargs):
        if kwargs.get('user_type') == 'admin':
            return func(kwargs)
        else:
            raise ValueError('Permission denied')
    return wrapper


@is_admin
def show_customer_receipt(user_type: str):
    very_important_data = {'token': 'qwerty123'} #  бажано не створювати змінних, тільки в з метою навчання
    very_important_data.clear()
    return print('Your data was deleted')

show_customer_receipt(user_type='admin')

Your data was deleted


2. Write a decorator that wraps a function in a try-except block and prints an error if any type of error has happened. 

Напишите декоратор, который оборачивает функцию в блок try-except и выводит ошибку, если произошла ошибка любого типа
Example:

@catch_errors
def some_function_with_risky_operation(data):
    print(data['key'])


some_function_with_risky_operation({'foo': 'bar'})
#Found 1 error during execution of your function: KeyError no such key as foo

some_function_with_risky_operation({'key': 'bar'})
#bar


In [104]:
def catch_errors(func):
    def wrapper(*args):
        try:
            return func(*args)
        except Exception as be:
            print(f'Found 1 error during execution of your function: {be}')      
    return wrapper

@catch_errors
def some_function_with_risky_operation(a, b):
    return a / b
    

some_function_with_risky_operation(1, 0)
some_function_with_risky_operation(1, 2)

Found 1 error during execution of your function: division by zero


0.5

Optional: Create a decorator that will check types. It should take a function with arguments and validate inputs with annotations. It should work for all possible functions. Don`t forget to check the return type as well

Дополнительно: Создайте декоратор, который будет проверять типы. Он должен принимать функцию с аргументами и проверять входные данные с помощью аннотаций. Он должен работать для всех возможных функций. Не забудьте также проверить возвращаемый тип



In [53]:
from functools import wraps

def type_check_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        annnotation_dict = func.__annotations__
        for arg_name, arg_type in annnotation_dict.items():
            if arg_name in kwargs:
                if not isinstance(kwargs[arg_name], arg_type):
                    raise TypeError(f"Argument '{arg_name}' must be '{arg_type.__name__}'")
            elif arg_name in annnotation_dict:
                arg_index = list(annnotation_dict.keys()).index(arg_name)
                if arg_index >= len(args) or not isinstance(args[arg_index], arg_type):
                    raise TypeError(f"Argument '{arg_name}' must be '{arg_type.__name__}'")
        result = func(*args, **kwargs)
        if 'return' in annnotation_dict and not isinstance(result, annnotation_dict['return']):
            raise TypeError(f"Return value must be '{annnotation_dict['return'].__name__}'")
        return result
    return wrapper

# Applying the decorator
@type_check_decorator
def add_numbers(a: int, b: int) -> int:
    return a + b




print(add_numbers(3, "4")) 



TypeError: Argument 'a' must be 'int'

Optional: Create a function that caches the result of a function, so that if it is called with the same argument multiple times, it returns the cached result first instead of re-executing the function

In [41]:

def cache_func_result(func):
    result_dict = {} 
    def wrapper(*args):
        if args in result_dict:
            return result_dict[args]
        else:
            result = func(*args)
            result_dict[args] = result
            return result
    return wrapper

@cache_func_result
def multiply_numbers(number): 
    return number * 1000000

print(multiply_numbers(10000))   

second call
10000000000


5. Optional: Write a decorator that adds a rate-limiter to a function, so that it can only be called a certain amount of times per minute
Напишите декоратор, который добавляет ограничитель скорости к функции, чтобы она могла вызываться только определенное количество раз в минуту

In [21]:
import time
from functools import wraps

def limit_func_calls(limit, seconds_interval):
    def wrapper_1(func):
        last_call = [0]
        calls_in_seconds = [0]
        @wraps(func)
        def wrapper_2(*args, **kwargs):
            current_time = time.time() # поточний час
            if current_time - last_call[0] >= seconds_interval: # Обнуляємо перемінну-лічильник calls_in_interval якщо з моменту останнього 
                                                                #виклику пройшов заданний інтервал часу
                calls_in_seconds[0] = 0
                last_call[0] = current_time
            if calls_in_seconds[0] < limit: # якщо не досягнуто ліміту викликів то викликаємо функцію та +1 до  calls_in_seconds
                calls_in_seconds[0] += 1
                return func(*args, **kwargs)
            else:
                print("Rate limit reached")       
        return wrapper_2  
    return wrapper_1


@limit_func_calls(5, 60)
def printing():
    print('doing something!')


for _ in range(10):
    printing()


doing something!
doing something!
doing something!
doing something!
doing something!
Rate limit reached
Rate limit reached
Rate limit reached
Rate limit reached
Rate limit reached


Type error: Argument 'return' should be of type 'int'
